In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats, anisotropy
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py



## Nihan's Data

In [ ]:
# decision threshold from the frequentist analysis for l = 0 through l = 6
freq_dthresh = np.array([1.        , 0.39622564, 0.32362566, 0.20984702, 0.11960839,
       0.07806638, 0.04426357])

# This is for the all-band Bayesian analysis from l = 1 to l = 6:
bayes_Cl = np.array([0.24187743, 0.17480158, 0.10009671, 0.08109598, 0.05493891,
       0.0317599 ])

# And these are the per-frequency Bayesian upper limits, 
# where each row represents the frequency bin from low to high (0 to 4) 
# and then each column is the spherical harmonic multipole from l = 1 to l = 6:
# (F, l)
Cl_nihan = np.array([
    [0.20216773, 0.14690035, 0.09676646, 0.07453352, 0.05500382, 0.03177427],
    [0.21201336, 0.14884939, 0.10545698, 0.07734305, 0.05257189, 0.03090662],
    [0.20840993, 0.14836757, 0.09854803, 0.07205384, 0.05409881, 0.03305785],
    [0.19788951, 0.15765126, 0.09615489, 0.07475364, 0.0527356 , 0.03113331],
    [0.20182648, 0.14745265, 0.09681202, 0.0746824 , 0.05503161, 0.0317012 ]])
print(Cl_nihan.shape)

freq_bins_Hz = np.array([2.0, 4.0, 5.9, 7.9, 9.9]) *10**-9 # Hz

# Functions

read_split_anis_file()

In [ ]:
def read_split2_anis_file(filename1, filename2):
    file1 = np.load(filename1)
    file2 = np.load(filename2)
    print(f"{file1.files=}")
    # load ss info
    shape = file1['ss_shape']
    nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
    fobs = file1['fobs']

    # # load ranking info
    # nsort = file1['nsort']
    # fidx = file1['fidx']
    # hc_ref = file1['hc_ref']

    # load harmonics info
    # nside = file1['nside']
    lmax  = file1['lmax']

    # load map and harmonics from 1st split chunk
    # moll_hc_best1 = file1['moll_hc_best']
    Cl_best1 = file1['Cl_best']
    nbest1 = len(Cl_best1)

    file1.close()

    # load map and harmonics from 2nd split chunk
    # moll_hc_best2 = file2['moll_hc_best']
    Cl_best2 = file2['Cl_best']
    nbest2 = len(Cl_best2)
    file2.close()


    # concatenate Cl and moll_hc
    nbest=nbest1+nbest2
    print(f"{nbest=}")
    Cl_best = np.concatenate((Cl_best1, Cl_best2)).reshape(nbest, nfreqs, nreals, lmax+1)
    # moll_hc_best = np.concatenate((moll_hc_best1, moll_hc_best2), axis=0).reshape(nbest, nfreqs, nreals, npix)

    print(f"{shape=}, {nbest=}")
    print(f"{Cl_best1.shape=}, {Cl_best2.shape=}, {Cl_best.shape=}")
    # print(f"{moll_hc_best1.shape=}, {moll_hc_best2.shape=}, {moll_hc_best.shape=}")

    vals = {
        'fobs':fobs, 'lmax':lmax, 'Cl_best':Cl_best
    }
    return vals


read_one_anis_file()

In [ ]:
def read_one_anis_file(filename1):
    file1 = np.load(filename1)
    print(f"{file1.files=}")
    # load ss info
    shape = file1['ss_shape']
    nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
    fobs = file1['fobs']

    # # load ranking info
    # nsort = file1['nsort']
    # fidx = file1['fidx']
    # hc_ref = file1['hc_ref']

    # load harmonics info
    # nside = file1['nside']
    lmax  = file1['lmax']

    # load map and harmonics from 1st split chunk
    # moll_hc_best1 = file1['moll_hc_best']
    Cl_best = file1['Cl_best']
    nbest = len(Cl_best)

    file1.close()


    print(f"{shape=}, {nbest=}")
    print(f"{Cl_best.shape=}")
    # print(f"{moll_hc_best1.shape=}, {moll_hc_best2.shape=}, {moll_hc_best.shape=}")

    vals = {
        'fobs':fobs, 'lmax':lmax, 'Cl_best':Cl_best
    }
    return vals


plot_ClC0_medians()

In [ ]:
def plot_ClC0_medians(xx, Cl_best, lmax, nshow, xx_Nihan, Cl_nihan,
                      figsize=(7,6), xlabel=plot.LABEL_GW_FREQUENCY_HZ, ylabel='$C_{\ell}/C_0$'):
    fig, ax = plot.figax(figsize=figsize, xlabel=xlabel, ylabel=ylabel,)

    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=-1) # (B,F,l) median over realizations

    # colors = ['k', 'b', 'r', 'g', 'c', 'm']
    colors = ['#0a5da4', '#00ba44', '#ff9503', '#fe2c02', '#845b98', '#474747']
    for ll in range(lmax):
        ax.plot(xx, np.median(yy[:,:,ll], axis=0), color=colors[ll]) #, label='median of samples, $l=%d$' % ll)
        for pp in [68,]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,:,ll], percs, axis=0), alpha=0.1, color=colors[ll])
        
        for bb in range(0,nshow):
            # if ll==0 and bb==0:
            #     label = "individual best samples, median of realizations"
            # else: 
            label=None
            ax.plot(xx, yy[bb,:,ll], color=colors[ll], linestyle=':', alpha=0.25,
                                 linewidth=1, label=label)
            
        # Add Nihan's data
        ax.plot(xx_Nihan, Cl_nihan[:,ll], 
                label = '$l=%d$' % (ll+1), 
            color=colors[ll], marker='o', ms=5)
        
        plot._twin_yr(ax, nano=False)
    ax.set_xlabel(xlabel, fontsize=11)
    ax.set_ylabel(ylabel, fontsize=11)
    # ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
    #             % (nbest, lmax))
    return fig
# fig1 = plot_ClC0_medians(fobs_yrs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz * YR, Cl_nihan=Cl_nihan)


plot_ClC0_medians_big

In [ ]:
def plot_ClC0_medians_big(xx, Cl_best, lmax, nshow, xx_Nihan, Cl_nihan,
                      xmin=None, xmax=None, ymin=None, ymax=None, show_spk = False):
    fig, ax = plot.figax(figsize=(7,6), xlabel=plot.LABEL_GW_FREQUENCY_HZ, ylabel='$C_{\ell}/C_0$')

    yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)
    yy = np.median(yy, axis=-1) # (B,F,l) median over realizations

    # colors = ['k', 'b', 'r', 'g', 'c', 'm']
    colors = ['#0a5da4', '#00ba44', '#ff9503', '#fe2c02', '#845b98', '#474747']
    for ll in range(lmax):
        ax.plot(xx, np.median(yy[:,:,ll], axis=0), color=colors[ll]) #, label='median of samples, $l=%d$' % ll)
        for pp in [50, 95]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[:,:,ll], percs, axis=0), alpha=0.1, color=colors[ll])
        
        for bb in range(0,nshow):
            # if ll==0 and bb==0:
            #     label = "individual best samples, median of realizations"
            # else: 
            label=None
            ax.plot(xx, yy[bb,:,ll], color=colors[ll], linestyle=':', alpha=0.25,
                                 linewidth=1, label=label)
            
        # Add Nihan's data
        ax.plot(xx_Nihan, Cl_nihan[:,ll], 
                label = '$l=%d$' % (ll+1), 
            color=colors[ll], marker='o', ms=8)
        
        plot._twin_yr(ax, nano=False)

    if show_spk:
        # Sato-Polito & Kamionkowski
        spk_xx = np.array([3.5*10**-9, 1.25*10**-8, 1*10**-7])
        spk_yy = np.array([1*10**-5, 1*10**-3, 1*10**-1])
        ax.plot(spk_xx * YR, spk_yy, label='SP & K', color='tab:orange')

    if xmin is not None and xmax is not None:
        ax.set_xlim(xmin, xmax)
    if ymin is not None and ymax is not None:
        ax.set_ylim(ymin, ymax)
    
    # ax.set_title('50%% and 98%% confidence intervals of the %d best samples \nusing realizations medians, lmax=%d'
    #             % (nbest, lmax))
    return fig
# fig1 = plot_ClC0_medians(fobs_yrs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz * YR, Cl_nihan=Cl_nihan)


# Uniform_09A

## 09A ns 32

In [ ]:
pspace = 'Uniform09A'
nside = 32

filename1 = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss58_anis09A_2023-06-15_uniform-09a_n1000_r50_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns32_r50_b00-49.npz'
filename2 = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss58_anis09A_2023-06-15_uniform-09a_n1000_r50_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns32_r50_b50-99.npz'

In [ ]:
vals = read_split2_anis_file(filename1, filename2)
fobs = vals['fobs']
lmax = vals['lmax']
Cl_best = vals['Cl_best']

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                        figsize=(7/1.65, 6/1.65))
ax = fig.axes[0]

ax.set_xlim(fobs[0]-10**-10, 1/YR)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10) 

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3, fontsize=8)

# fig.tight_layout()
# fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/modelhc2dOm_ClC0_ns%d.png' % nside, dpi=300)
# fig.text(0.05,1, 'nside=%d' % nside)
fig.tight_layout()

In [ ]:
fig = plot_ClC0_medians_big(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                            show_spk=True, xmin = fobs[0]-10**-10, xmax = 1/YR, ymin = 10**-4, ymax = 1.5)
ax = fig.axes[0]
ax.legend(loc='lower right', ncols=4)

fig.text(0.05,1, 'nside=%d, pspace=%s' % (nside, pspace), fontsize=14)
fig.tight_layout()

## 09a ns 08

In [ ]:
pspace = 'Uniform09A'
nside = 8

filename1 = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss58_anis09A_2023-06-15_uniform-09a_n1000_r50_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns08_r50_b00-99.npz'

In [ ]:
vals = read_one_anis_file(filename1)
fobs = vals['fobs']
lmax = vals['lmax']
Cl_best = vals['Cl_best']

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                        figsize=(7/1.65, 6/1.65))
ax = fig.axes[0]

ax.set_xlim(fobs[0]-10**-10, 1/YR)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10) 

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3, fontsize=8)

# fig.tight_layout()
# fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/modelhc2dOm_ClC0_ns%d.png' % nside, dpi=300)
# fig.text(0.05,1, 'nside=%d' % nside)
fig.tight_layout()

In [ ]:
fig = plot_ClC0_medians_big(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                            show_spk=True, xmin = fobs[0]-10**-10, xmax = 1/YR, ymin = 10**-4, ymax = 1.5)
ax = fig.axes[0]
ax.legend(loc='lower right', ncols=4)

fig.text(0.05,1, 'nside=%d, pspace=%s' % (nside, pspace), fontsize=14)
fig.tight_layout()

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

# Uniform_07A

## 07a nside 32

In [ ]:
pspace = 'Uniform07A'
nside = 32

filename1 = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns32_r50_b00-49.npz'
filename2 = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns32_r50_b50-99.npz'

vals = read_split2_anis_file(filename1, filename2)

In [ ]:
fobs = vals['fobs']
lmax = vals['lmax']
Cl_best = vals['Cl_best']

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                        figsize=(7/1.65, 6/1.65))
ax = fig.axes[0]

ax.set_xlim(fobs[0]-10**-10, 1/YR)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10) 

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3, fontsize=8)

# fig.tight_layout()
# fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/modelhc2dOm_ClC0_ns%d.png' % nside, dpi=300)
# fig.text(0.05,1, 'nside=%d' % nside)
fig.tight_layout()

In [ ]:
fig = plot_ClC0_medians_big(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                            show_spk=True, xmin = fobs[0]-10**-10, xmax = 1/YR, ymin = 10**-4, ymax = 1.5)
ax = fig.axes[0]
ax.legend(loc='lower right', ncols=4)

fig.text(0.05,1, 'nside=%d, pspace=%s' % (nside, pspace), fontsize=14)
fig.tight_layout()

## 07A nside 08

In [ ]:
pspace = 'Uniform07A'
nside = 8

filename1 = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns08_r50_b00-49.npz'
filename2 = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/anisotropy/sph_harm_hc2dOm_lmax6_ns08_r50_b50-99.npz'

vals = read_split2_anis_file(filename1, filename2)

In [ ]:
fobs = vals['fobs']
lmax = vals['lmax']
Cl_best = vals['Cl_best']

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                        figsize=(7/1.65, 6/1.65))
ax = fig.axes[0]

ax.set_xlim(fobs[0]-10**-10, 1/YR)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10) 

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3, fontsize=8)

# fig.tight_layout()
# fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/modelhc2dOm_ClC0_ns%d.png' % nside, dpi=300)
# fig.text(0.05,1, 'nside=%d' % nside)
fig.tight_layout()

In [ ]:
fig = plot_ClC0_medians_big(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                            show_spk=True, xmin = fobs[0]-10**-10, xmax = 1/YR, ymin = 10**-4, ymax = 1.5)
ax = fig.axes[0]
ax.legend(loc='lower right', ncols=4)

fig.text(0.05,1, 'nside=%d, pspace=%s' % (nside, pspace), fontsize=14)
fig.tight_layout()

# Uniform_05A

## 05A ns 08

In [ ]:
pspace = 'Uniform_05A'
nside = 32

filename1 = '/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss19_uniform05A_n1000_r50_d20_f30_l2000_p0/anisotropy/sph_harm_hc2dOm_lmax6_ns08_r50_b00-49.npz'
filename2 = '/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss19_uniform05A_n1000_r50_d20_f30_l2000_p0/anisotropy/sph_harm_hc2dOm_lmax6_ns08_r50_b50-99.npz'

vals = read_split2_anis_file(filename1, filename2)

In [ ]:
fobs = vals['fobs']
lmax = vals['lmax']
Cl_best = vals['Cl_best']

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                        figsize=(7/1.65, 6/1.65))
ax = fig.axes[0]

ax.set_xlim(fobs[0]-10**-10, 1/YR)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10) 

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3, fontsize=8)

# fig.tight_layout()
# fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/modelhc2dOm_ClC0_ns%d.png' % nside, dpi=300)
# fig.text(0.05,1, 'nside=%d' % nside)
fig.tight_layout()

In [ ]:
fig = plot_ClC0_medians_big(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                            show_spk=True, xmin = fobs[0]-10**-10, xmax = 1/YR, ymin = 10**-4, ymax = 1.5)
ax = fig.axes[0]
ax.legend(loc='lower right', ncols=4)

fig.text(0.05,1, 'nside=%d, pspace=%s' % (nside, pspace), fontsize=14)
fig.tight_layout()

# Uniform07_GW

## 07GW ns 32

In [ ]:
pspace = 'Uniform09_GW'
nside = 32

filename1 = 
filename2 = 

vals = read_split2_anis_file(filename1, filename2)

In [ ]:
fobs = vals['fobs']
lmax = vals['lmax']
Cl_best = vals['Cl_best']

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                        figsize=(7/1.65, 6/1.65))
ax = fig.axes[0]

ax.set_xlim(fobs[0]-10**-10, 1/YR)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10) 

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3, fontsize=8)

# fig.tight_layout()
# fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/modelhc2dOm_ClC0_ns%d.png' % nside, dpi=300)
# fig.text(0.05,1, 'nside=%d' % nside)
fig.tight_layout()

In [ ]:
fig = plot_ClC0_medians_big(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                            show_spk=True, xmin = fobs[0]-10**-10, xmax = 1/YR, ymin = 10**-4, ymax = 1.5)
ax = fig.axes[0]
ax.legend(loc='lower right', ncols=4)

fig.text(0.05,1, 'nside=%d, pspace=%s' % (nside, pspace), fontsize=14)
fig.tight_layout()

## 07GW ns 08

In [ ]:
pspace = 'Uniform09_GW'
nside = 8

filename1 = 
filename2 = 

vals = read_split2_anis_file(filename1, filename2)

In [ ]:
fobs = vals['fobs']
lmax = vals['lmax']
Cl_best = vals['Cl_best']

### Plot Results

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR
ff_labels = (0,9,19,29,39)
nshow = 10

In [ ]:
fig = plot_ClC0_medians(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                        figsize=(7/1.65, 6/1.65))
ax = fig.axes[0]

ax.set_xlim(fobs[0]-10**-10, 1/YR)   # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10) 

# fig1.legend(bbox_to_anchor=(0,0), loc='upper left', bbox_transform=ax.transAxes)
ax.legend(loc='lower right', ncols=3, fontsize=8)

# fig.tight_layout()
# fig.savefig('/Users/emigardiner/GWs/holodeck/ecg-notebooks/anisotropy/modelhc2dOm_ClC0_ns%d.png' % nside, dpi=300)
# fig.text(0.05,1, 'nside=%d' % nside)
fig.tight_layout()

In [ ]:
fig = plot_ClC0_medians_big(fobs, Cl_best, lmax, nshow=10, xx_Nihan=freq_bins_Hz, Cl_nihan=Cl_nihan,
                            show_spk=True, xmin = fobs[0]-10**-10, xmax = 1/YR, ymin = 10**-4, ymax = 1.5)
ax = fig.axes[0]
ax.legend(loc='lower right', ncols=4)

fig.text(0.05,1, 'nside=%d, pspace=%s' % (nside, pspace), fontsize=14)
fig.tight_layout()